In [32]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

In [33]:
# Load and combine data
df_2023 = pd.read_csv('survey_results_public_2023.csv')
df_2024 = pd.read_csv('survey_results_public_2024.csv')
df_2023['Year'] = 2023
df_2024['Year'] = 2024
df = pd.concat([df_2023, df_2024], ignore_index=True)

In [34]:
# Handle missing salaries
df = df.dropna(subset=['ConvertedCompYearly'])
print(f"Number of rows after dropping missing salaries: {len(df)}")

Number of rows after dropping missing salaries: 71454


In [35]:
# Clean salary outliers (relaxed range)
df = df[(df['ConvertedCompYearly'] >= 10000) & (df['ConvertedCompYearly'] <= 1000000)]
df['LogSalary'] = np.log1p(df['ConvertedCompYearly'])
print(f"Number of rows after filtering outliers: {len(df)}")

Number of rows after filtering outliers: 65725


/var/folders/x3/82hmpcmd3t7c9vvrdghydg8m0000gn/T/ipykernel_37340/2291080988.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['LogSalary'] = np.log1p(df['ConvertedCompYearly'])


In [36]:
# Clean YearsCode and YearsCodePro
def clean_years_column(series):
    series = series.replace({'Less than 1 year': '0', 'More than 50 years': '50'})
    series = pd.to_numeric(series, errors='coerce')
    median_value = series.median()
    series = series.fillna(median_value)
    return series

df['YearsCode'] = clean_years_column(df['YearsCode'])
df['YearsCodePro'] = clean_years_column(df['YearsCodePro'])

# Create ExperienceLevel
def create_experience_level(years):
    if years < 3:
        return 'Junior'
    elif years <= 7:
        return 'Mid'
    else:
        return 'Senior'

df['ExperienceLevel'] = df['YearsCodePro'].apply(create_experience_level)

/var/folders/x3/82hmpcmd3t7c9vvrdghydg8m0000gn/T/ipykernel_37340/2115734665.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['ExperienceLevel'] = df['YearsCodePro'].apply(create_experience_level)


In [37]:
# Simplify DevType
def simplify_dev_type(dev_type):
    if pd.isna(dev_type):
        return 'Unknown'
    dev_type = dev_type.lower()
    if 'full-stack' in dev_type:
        return 'Full-stack developer'
    if 'back-end' in dev_type:
        return 'Back-end developer'
    if 'front-end' in dev_type:
        return 'Front-end developer'
    if 'data scientist' in dev_type or 'data engineer' in dev_type:
        return 'Data scientist'
    if 'devops' in dev_type:
        return 'DevOps engineer'
    return 'Other'

if 'DevType' in df.columns:
    df['DevType'] = df['DevType'].apply(simplify_dev_type)


In [38]:
# Simplify EdLevel
def simplify_ed_level(ed_level):
    if pd.isna(ed_level):
        return 'Unknown'
    ed_level = ed_level.lower()
    if 'bachelor' in ed_level:
        return 'Bachelor’s degree'
    if 'master' in ed_level:
        return 'Master’s degree'
    if 'doctoral' in ed_level or 'phd' in ed_level:
        return 'Doctoral degree'
    if 'associate' in ed_level:
        return 'Associate degree'
    return 'Other'

if 'EdLevel' in df.columns:
    df['EdLevel'] = df['EdLevel'].apply(simplify_ed_level)

In [39]:
# Simplify Employment
def simplify_employment(employment):
    if pd.isna(employment):
        return 'Unknown'
    employment = employment.lower()
    if 'full-time' in employment:
        return 'Employed, full-time'
    if 'freelancer' in employment or 'contractor' in employment:
        return 'Freelancer'
    if 'part-time' in employment:
        return 'Part-time'
    return 'Other'

if 'Employment' in df.columns:
    df['Employment'] = df['Employment'].apply(simplify_employment)

In [40]:

# Feature engineering
df['NumLanguages'] = df['LanguageHaveWorkedWith'].str.split(';').apply(lambda x: len(x) if isinstance(x, list) else 0)
df['NumTools'] = df['ToolsTechHaveWorkedWith'].str.split(';').apply(lambda x: len(x) if isinstance(x, list) else 0)
df['NumPlatforms'] = df['PlatformHaveWorkedWith'].str.split(';').apply(lambda x: len(x) if isinstance(x, list) else 0)
df['YearsCodePro_DevType'] = df['YearsCodePro'].astype(str) + '_' + df['DevType']
df['Country_EdLevel'] = df['Country'] + '_' + df['EdLevel']



/var/folders/x3/82hmpcmd3t7c9vvrdghydg8m0000gn/T/ipykernel_37340/3106874623.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['NumLanguages'] = df['LanguageHaveWorkedWith'].str.split(';').apply(lambda x: len(x) if isinstance(x, list) else 0)
/var/folders/x3/82hmpcmd3t7c9vvrdghydg8m0000gn/T/ipykernel_37340/3106874623.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['NumTools'] = df['ToolsTechHaveWorkedWith'].str.split(';').apply(lambda x: len(x) if isinstance(x, list) else 0)
/var/folders/x3/82hmpcmd3t7c9vvrdghydg8m000

In [41]:
# Interaction terms
df['YearsCodePro_DevType'] = df['YearsCodePro'].astype(str) + '_' + df['DevType']

In [42]:
# Multi-select features
new_columns = []
if 'LanguageHaveWorkedWith' in df.columns:
    languages = df['LanguageHaveWorkedWith'].str.get_dummies(';')
    languages = languages[[col for col in languages.columns if col != 'Apples' and languages[col].sum() > 100]]
    languages.columns = [f'Language_{col}' for col in languages.columns]
    new_columns.append(languages)

if 'DatabaseHaveWorkedWith' in df.columns:
    databases = df['DatabaseHaveWorkedWith'].str.get_dummies(';')
    databases = databases[[col for col in databases.columns if databases[col].sum() > 100]]
    databases.columns = [f'Database_{col}' for col in databases.columns]
    new_columns.append(databases)

if 'PlatformHaveWorkedWith' in df.columns:
    platforms = df['PlatformHaveWorkedWith'].str.get_dummies(';')
    platforms = platforms[[col for col in platforms.columns if platforms[col].sum() > 100]]
    platforms.columns = [f'Platform_{col}' for col in platforms.columns]
    new_columns.append(platforms)

# Concatenate all new columns
if new_columns:
    df = pd.concat([df] + new_columns, axis=1, copy=True)

In [43]:
# Select categorical and numerical features
exclude_columns = ['ConvertedCompYearly', 'CompTotal', 'Currency', 'ResponseId', 
                   'LanguageHaveWorkedWith', 'DatabaseHaveWorkedWith', 
                   'PlatformHaveWorkedWith', 'ToolsTech', 'WebframeHaveWorkedWith', 
                   'LogSalary']
categorical_features = [col for col in df.select_dtypes(include=['object', 'category']).columns 
                        if col not in exclude_columns and df[col].nunique() < 100]
preferred_cats = ['EdLevel', 'Country', 'RemoteWork', 'DevType', 'Employment', 
                 'OrgSize', 'Industry', 'ExperienceLevel', 'YearsCodePro_DevType', 'Country_EdLevel']
categorical_features = preferred_cats + [col for col in categorical_features 
                                        if col not in preferred_cats and col.endswith('_Simplified')]
categorical_features = [col for col in categorical_features if col in df.columns]
for col in categorical_features:
    df[col].fillna('Unknown', inplace=True)

numerical_features = ['NumLanguages', 'NumTools', 'NumPlatforms', 'YearsCode', 'YearsCodePro'] + \
                     [col for col in df.columns if col.startswith('Language_') or 
                      col.startswith('Database_') or col.startswith('Platform_')]

# Handle NaN/inf in numerical features
for col in numerical_features:
    if df[col].isna().any() or np.isinf(df[col]).any():
        print(f"Warning: {col} contains NaN or inf values")
        df[col] = df[col].replace([np.inf, -np.inf], np.nan).fillna(df[col].median())

# Filter zero-variance features
numerical_features = [col for col in numerical_features if df[col].std() > 0]
print("Filtered Numerical Features:", numerical_features)
print("Categorical Features:", categorical_features)

# Print feature distributions
for col in ['DevType', 'EdLevel', 'Employment', 'YearsCodePro_DevType', 'Country_EdLevel']:
    if col in df.columns:
        print(f"{col} distribution:\n", df[col].value_counts().head(10))

Filtered Numerical Features: ['NumLanguages', 'NumTools', 'NumPlatforms', 'YearsCode', 'YearsCodePro', 'Language_Ada', 'Language_Apex', 'Language_Assembly', 'Language_Bash/Shell (all shells)', 'Language_C', 'Language_C#', 'Language_C++', 'Language_Clojure', 'Language_Cobol', 'Language_Crystal', 'Language_Dart', 'Language_Delphi', 'Language_Elixir', 'Language_Erlang', 'Language_F#', 'Language_Flow', 'Language_Fortran', 'Language_GDScript', 'Language_Go', 'Language_Groovy', 'Language_HTML/CSS', 'Language_Haskell', 'Language_Java', 'Language_JavaScript', 'Language_Julia', 'Language_Kotlin', 'Language_Lisp', 'Language_Lua', 'Language_MATLAB', 'Language_MicroPython', 'Language_Nim', 'Language_OCaml', 'Language_Objective-C', 'Language_PHP', 'Language_Perl', 'Language_PowerShell', 'Language_Prolog', 'Language_Python', 'Language_R', 'Language_Ruby', 'Language_Rust', 'Language_SAS', 'Language_SQL', 'Language_Scala', 'Language_Solidity', 'Language_Swift', 'Language_TypeScript', 'Language_VBA', '

/var/folders/x3/82hmpcmd3t7c9vvrdghydg8m0000gn/T/ipykernel_37340/1982447666.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('Unknown', inplace=True)


In [44]:
new_df = df.copy()
#write to CSV with 5000 rows
new_df = new_df.sample(n=5000, random_state=42)
new_df.to_csv('processed_survey_data.csv', index=False)

In [45]:
# Define features and target
X = df[numerical_features + categorical_features]
y = df['LogSalary']

# Check for NaN/inf in target
if y.isna().any() or np.isinf(y).any():
    print("Warning: LogSalary contains NaN or inf values")
    valid_mask = ~(y.isna() | np.isinf(y))
    X = X[valid_mask]
    y = y[valid_mask]

In [46]:
# check null values
print("Null values in features:\n", X.isnull().sum())

Null values in features:
 NumLanguages            0
NumTools                0
NumPlatforms            0
YearsCode               0
YearsCodePro            0
                       ..
OrgSize                 0
Industry                0
ExperienceLevel         0
YearsCodePro_DevType    0
Country_EdLevel         0
Length: 125, dtype: int64


In [47]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
# Define preprocessing

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ],
)



In [49]:
# Apply SMOTE (optional, enable after fixing versions)
# try:
    # from imblearn.over_sampling import SMOTE
    # smote = SMOTE(random_state=42)
    # X_train_preprocessed = preprocessor.fit_transform(X_train)
    # X_train_smote, y_train_smote = smote.fit_resample(X_train_preprocessed, y_train)
    # X_train_final, y_train_final = X_train_smote, y_train_smote
    # print("SMOTE applied successfully")
# except ImportError as e:
    # print(f"SMOTE ImportError: {e}. Using original training data.")
X_train_final, y_train_final = preprocessor.fit_transform(X_train), y_train

In [50]:
# Get one-hot encoded feature names for key categorical features
key_cats = ['DevType', 'EdLevel', 'Employment', 'YearsCodePro_DevType', 'Country_EdLevel']
key_cat_features = []
temp_preprocessor = preprocessor.fit(X_train)
feature_names = temp_preprocessor.get_feature_names_out()
for cat in key_cats:
    key_cat_features.extend([f for f in feature_names if f.startswith(f'cat__{cat}_')])
print("Key Categorical Features:", key_cat_features)

Key Categorical Features: ['cat__DevType_Back-end developer', 'cat__DevType_Data scientist', 'cat__DevType_DevOps engineer', 'cat__DevType_Front-end developer', 'cat__DevType_Full-stack developer', 'cat__DevType_Other', 'cat__DevType_Unknown', 'cat__EdLevel_Associate degree', 'cat__EdLevel_Bachelor’s degree', 'cat__EdLevel_Master’s degree', 'cat__EdLevel_Other', 'cat__Employment_Employed, full-time', 'cat__Employment_Freelancer', 'cat__Employment_Other', 'cat__Employment_Part-time', 'cat__Employment_Unknown', 'cat__YearsCodePro_DevType_0.0_Back-end developer', 'cat__YearsCodePro_DevType_0.0_Data scientist', 'cat__YearsCodePro_DevType_0.0_DevOps engineer', 'cat__YearsCodePro_DevType_0.0_Front-end developer', 'cat__YearsCodePro_DevType_0.0_Full-stack developer', 'cat__YearsCodePro_DevType_0.0_Other', 'cat__YearsCodePro_DevType_0.0_Unknown', 'cat__YearsCodePro_DevType_1.0_Back-end developer', 'cat__YearsCodePro_DevType_1.0_Data scientist', 'cat__YearsCodePro_DevType_1.0_DevOps engineer', 

In [51]:
# Define models
models = {
    'LinearRegression': {
        'model': LinearRegression(),
        'param_grid': {
            'model__fit_intercept': [True, False]
        }
    },
    # 'RandomForest': {
    #     'model': RandomForestRegressor(random_state=42),
    #     'param_grid': {
    #         'model__n_estimators': [100, 200, 300],
    #         'model__max_depth': [10, 15, 20, None],
    #         'model__min_samples_split': [2, 5]
    #     }
    # },
    # 'XGBoost': {
        # 'model': XGBRegressor(random_state=42),
        # 'param_grid': {
            # 'model__n_estimators': [100, 200, 300, 400],
            # 'model__max_depth': [3, 5, 7, 9],
            # 'model__learning_rate': [0.01, 0.05, 0.1, 0.2],
            # 'model__reg_lambda': [0.1, 1.0, 5.0],
            # 'model__subsample': [0.8, 1.0]
        # }
    # },
    # 'Stacking': {
        # 'model': StackingRegressor(
            # estimators=[
                # ('rf', RandomForestRegressor(random_state=42)),
                # ('xgb', XGBRegressor(random_state=42))
            # ],
            # final_estimator=LinearRegression()
        # ),
        # 'param_grid': {
            # 'model__rf__n_estimators': [100, 200],
            # 'model__rf__max_depth': [10, 15],
            # 'model__xgb__n_estimators': [100, 200],
            # 'model__xgb__max_depth': [3, 5],
            # 'model__xgb__learning_rate': [0.05, 0.1]
        # }
    # }
}

In [52]:
# Evaluate models
best_models = {}
results = {}

for model_name, config in models.items():
    print(f"\nTraining {model_name}...")
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('select_kbest', SelectKBest(score_func=f_regression, k=30)),
        ('feature_selection', SelectFromModel(XGBRegressor(random_state=42), max_features=20, threshold='median')),
        ('model', config['model'])
    ])
    
    grid_search = GridSearchCV(
        pipeline, 
        config['param_grid'], 
        cv=5, 
        scoring='neg_mean_squared_error', 
        n_jobs=-1
    )
    grid_search.fit(X_train, y_train)
    
    # Best model
    best_models[model_name] = grid_search.best_estimator_
    print(f"Best Parameters for {model_name}:", grid_search.best_params_)
    
    # Ensure key categorical features
    feature_mask = grid_search.best_estimator_.named_steps['feature_selection'].get_support()
    kbest_mask = grid_search.best_estimator_.named_steps['select_kbest'].get_support()
    feature_names = grid_search.best_estimator_.named_steps['preprocessor'].get_feature_names_out()
    selected_features = feature_names[kbest_mask][feature_mask].tolist()
    for key_feature in key_cat_features:
        if key_feature not in selected_features:
            selected_features.append(key_feature)
            if len(selected_features) > 20:
                selected_features = selected_features[:20]
    print(f"Selected Features for {model_name}:", selected_features)
    
    # Evaluate on test set
    y_pred = grid_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(np.expm1(y_test), np.expm1(y_pred))
    r2 = r2_score(y_test, y_pred)
    train_score = grid_search.best_estimator_.score(X_train, y_train)
    test_score = grid_search.best_estimator_.score(X_test, y_test)
    
    results[model_name] = {
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'Training Score': train_score,
        'Test Score': test_score
    }
    feature_names = grid_search.best_estimator_.named_steps['preprocessor'].get_feature_names_out()
    print(f"Feature Names for {model_name}:", feature_names)
    print(f"Results for {model_name}:\n", results[model_name])
    # Save model and features
    joblib.dump(grid_search.best_estimator_, f'models/{model_name}_salary_predictor.pkl')
    joblib.dump(key_cat_features, f'models/{model_name}_feature_names.pkl')


Training LinearRegression...


/Users/charles/Projects/Developers-Salary-Predictor/salary_predictor/venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/charles/Projects/Developers-Salary-Predictor/salary_predictor/venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/charles/Projects/Developers-Salary-Predictor/salary_predictor/venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/charles/.pyenv/versions/3.12.11/lib/python3.12/site-packages/numpy/linalg/_linalg.py:2791: RuntimeWarning: overflow encountered in dot
  sqnorm = x.dot(x)
/Users/charles/Projects/Developers-Salary-Predictor/salary_predictor/venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/charles/Projects/Developers-Salary-Pre

Best Parameters for LinearRegression: {'model__fit_intercept': True}
Selected Features for LinearRegression: ['num__YearsCode', 'num__YearsCodePro', 'num__Language_Go', 'num__Language_PHP', 'num__Database_Dynamodb', 'num__Database_MySQL', 'num__Database_Snowflake', 'num__Platform_Amazon Web Services (AWS)', 'cat__Country_Brazil', 'cat__Country_India', 'cat__Country_Italy', 'cat__Country_Ukraine', 'cat__Country_United States of America', 'cat__RemoteWork_In-person', 'cat__OrgSize_10,000 or more employees', 'cat__DevType_Back-end developer', 'cat__DevType_Data scientist', 'cat__DevType_DevOps engineer', 'cat__DevType_Front-end developer', 'cat__DevType_Full-stack developer']


/Users/charles/Projects/Developers-Salary-Predictor/salary_predictor/venv/lib/python3.12/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/charles/Projects/Developers-Salary-Predictor/salary_predictor/venv/lib/python3.12/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/charles/Projects/Developers-Salary-Predictor/salary_predictor/venv/lib/python3.12/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


Feature Names for LinearRegression: ['num__NumLanguages' 'num__NumTools' 'num__NumPlatforms' ...
 'cat__Country_EdLevel_Zimbabwe_Bachelor’s degree'
 'cat__Country_EdLevel_Zimbabwe_Master’s degree'
 'cat__Country_EdLevel_Zimbabwe_Other']
Results for LinearRegression:
 {'RMSE': np.float64(0.5341923718436694), 'MAE': 35601.032816470615, 'R2': 0.4728810420279629, 'Training Score': 0.4653240337464317, 'Test Score': 0.4728810420279629}


/Users/charles/Projects/Developers-Salary-Predictor/salary_predictor/venv/lib/python3.12/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/charles/Projects/Developers-Salary-Predictor/salary_predictor/venv/lib/python3.12/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/charles/Projects/Developers-Salary-Predictor/salary_predictor/venv/lib/python3.12/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
/Users/charles/Projects/Developers-Salary-Predictor/salary_predictor/venv/lib/python3.12/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/charles/Projects/Developers-Salary-Predictor/salary_predictor/venv/lib/python3.12/site-p

In [53]:
print("\nModel Comparison:")
for model_name, metrics in results.items():
    print(f"{model_name}:")
    print(f"  RMSE: {metrics['RMSE']:.4f}")
    print(f"  MAE: ${metrics['MAE']:.2f}")
    print(f"  R²: {metrics['R2']:.4f}")
    print(f"  Training Score: {metrics['Training Score']:.4f}")
    print(f"  Test Score: {metrics['Test Score']:.4f}")

# Select best model (highest R²)
best_model_name = max(results, key=lambda x: results[x]['R2'])
print(f"\nBest Model: {best_model_name}")


Model Comparison:
LinearRegression:
  RMSE: 0.5342
  MAE: $35601.03
  R²: 0.4729
  Training Score: 0.4653
  Test Score: 0.4729

Best Model: LinearRegression
